In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Eric Meinhardt / emeinhardt@ucsd.edu**

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Motivation" data-toc-modified-id="Motivation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Motivation</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#Combinatoric-summary" data-toc-modified-id="Combinatoric-summary-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Combinatoric summary</a></span></li></ul></li><li><span><a href="#Identifying-a-possible-Boolean-concept" data-toc-modified-id="Identifying-a-possible-Boolean-concept-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Identifying a possible Boolean concept</a></span></li><li><span><a href="#General-case" data-toc-modified-id="General-case-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>General case</a></span><ul class="toc-item"><li><span><a href="#Demo" data-toc-modified-id="Demo-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Demo</a></span></li></ul></li></ul></div>

# Motivation

In this notebook we show `prague` can be used to identify feature vectors compatible with (or that exactly pick out) an observed set of objects, using phonology as a motivating example.

Note that on the current test dataset (`brh.py`), the machine running this notebook will probably need (at peak) 20-30GB of RAM.

# Load data

In [2]:
%cd ..

/mnt/cube/home/AD/emeinhar/prague


In [3]:
from funcy import *
from functools import reduce

In [4]:
import numpy as np

In [5]:
import prague

In [6]:
%ls data

bakovic_chart_riggle_hayes_remapped.tsv  brh.npy             hayes_remapped.tsv
bakovic_chart_riggle_hayes.tsv           hayes_features.txt  hayes.tsv
brh_features.txt                         hayes.npy           mielke_spe.tsv


In [7]:
# objects_in_fp = 'data/hayes.tsv'
# objects_np_in_fp = 'data/hayes.npy'
# feature_list_fp = 'data/hayes_features.txt'

objects_in_fp = 'data/bakovic_chart_riggle_hayes.tsv'
objects_np_in_fp = 'data/brh.npy'
feature_list_fp = 'data/brh_features.txt'

In [8]:
objects = prague.load_objects(objects_in_fp)
len(objects)
objects[:3]

symbols = [o['symbol'] for o in objects]
symbols[:10]

94

[OrderedDict([('symbol', 'p'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '-'),
              ('cont', '-'),
              ('del. rel.', '-'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '0'),
              ('low', '0'),
              ('back', '0'),
              ('front', '0'),
              ('approx', '-'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '0'),
              ('ATR', '0')]),
 OrderedDict([('symbol', 'b'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '+'),
              ('cont', '-'),
              ('de

['p', 'b', 'ɸ', 'β', 'f', 'v', 't̪', 'd̪', 'θ', 'ð']

In [9]:
len(objects[0].keys())

24

In [10]:
feature_list = []
with open(feature_list_fp, 'r') as feature_file:
    for feature in feature_file:
        feature_list.append(feature.strip())
len(feature_list)

23

In [11]:
assert len(feature_list) == len(objects[0].keys()) - 1

In [12]:
objects_np = np.load(objects_np_in_fp)
objects_np.shape
objects_np.dtype

(94, 23)

dtype('int8')

In [13]:
assert objects_np.shape[1] == len(feature_list)

In [14]:
assert objects_np.shape[0] == len(objects)

In [15]:
objects[:2]

[OrderedDict([('symbol', 'p'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '-'),
              ('cont', '-'),
              ('del. rel.', '-'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '0'),
              ('low', '0'),
              ('back', '0'),
              ('front', '0'),
              ('approx', '-'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '0'),
              ('ATR', '0')]),
 OrderedDict([('symbol', 'b'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '+'),
              ('cont', '-'),
              ('de

In [16]:
objects_np[:2]

array([[ 0,  0, -1,  0, -1,  1, -1, -1, -1,  0, -1,  0,  0,  1, -1,  0,
        -1,  0, -1, -1,  0, -1, -1],
       [ 0,  0, -1,  0, -1,  1, -1, -1, -1,  0, -1,  0,  0,  1, -1,  0,
        -1,  0, -1, -1,  0, -1,  1]], dtype=int8)

In [17]:
symbol_to_fv = {o['symbol']:objects_np[i]
                for i,o in enumerate(objects)}

In [18]:
objects_np.shape
unique_objects_np = np.unique(objects_np, axis=0)
unique_objects_np.shape

(94, 23)

(91, 23)

In [19]:
fv_to_symbols_map = {prague.HashableArray(fv):{s for s in symbol_to_fv
                                               if np.array_equal(fv, symbol_to_fv[s])}
                     for fv in unique_objects_np}

def fv_to_symbols(fv):
    return fv_to_symbols_map[prague.HashableArray(fv)]

In [20]:
index_to_symbols = [fv_to_symbols(fv) for fv in unique_objects_np]
index_to_symbols

[{'ħ'},
 {'ʕ'},
 {'ə', 'ɜ'},
 {'ɞ'},
 {'ɛ'},
 {'a'},
 {'œ'},
 {'ɶ'},
 {'ɪ'},
 {'ʏ'},
 {'ʌ'},
 {'ɑ'},
 {'ɔ'},
 {'ɒ'},
 {'ʊ'},
 {'ʈ'},
 {'ɖ'},
 {'ɳ'},
 {'ʧ'},
 {'ʤ'},
 {'ʂ'},
 {'ʐ'},
 {'ʃ'},
 {'ʒ'},
 {'ɽ'},
 {'ɭ'},
 {'c'},
 {'ɟ'},
 {'ɲ'},
 {'ç'},
 {'ʝ'},
 {'h'},
 {'ɦ'},
 {'p'},
 {'b'},
 {'m', 'ɱ'},
 {'f'},
 {'v'},
 {'ɸ'},
 {'β'},
 {'ʔ'},
 {'q'},
 {'ɢ'},
 {'k'},
 {'ɡ'},
 {'ɴ'},
 {'ŋ'},
 {'χ'},
 {'ʁ'},
 {'x'},
 {'ɣ'},
 {'j'},
 {'ʎ'},
 {'ʋ'},
 {'ⱱ'},
 {'ʙ'},
 {'w', 'ɥ'},
 {'ʟ'},
 {'ʀ'},
 {'t'},
 {'d'},
 {'t̪'},
 {'d̪'},
 {'n'},
 {'n̪'},
 {'ʦ'},
 {'ʣ'},
 {'s'},
 {'z'},
 {'θ'},
 {'ð'},
 {'ɹ'},
 {'ɻ'},
 {'ɾ'},
 {'l'},
 {'l̪'},
 {'r'},
 {'ɘ'},
 {'ɐ'},
 {'ɵ'},
 {'ɨ'},
 {'ʉ'},
 {'e'},
 {'æ'},
 {'ø'},
 {'i'},
 {'y'},
 {'ɤ'},
 {'o'},
 {'ɯ'},
 {'u'}]

In [21]:
def extension_to_symbols(extension):
    return np.array(index_to_symbols)[extension.nonzero()[0]]

In [22]:
def symbol_to_fd(symbol):
    return [fd for fd in objects if fd['symbol'] == symbol]

symbol_to_fd('i')

[OrderedDict([('symbol', 'i'),
              ('syll', '+'),
              ('cons', '-'),
              ('son', '+'),
              ('labial', '-'),
              ('coronal', '-'),
              ('dorsal', '+'),
              ('voice', '+'),
              ('cont', '+'),
              ('del. rel.', '0'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '+'),
              ('low', '-'),
              ('back', '-'),
              ('front', '+'),
              ('approx', '+'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '-'),
              ('ATR', '+')])]

In [23]:
feature_list

['ATR',
 'ant',
 'approx',
 'back',
 'c.g.',
 'cons',
 'cont',
 'coronal',
 'del. rel.',
 'dist',
 'dorsal',
 'front',
 'high',
 'labial',
 'lat',
 'low',
 'nas',
 'round',
 's.g.',
 'son',
 'strid',
 'syll',
 'voice']

In [24]:
def pfv_to_fd(pfv):
    return prague.feature_vector.to_feature_dict(feature_list, pfv)

random_fd = pfv_to_fd(prague.feature_vector.make_random_pfv(len(feature_list)))
random_fd
random_spe = prague.to_spe(d=random_fd)
random_spe
prague.from_spe(random_spe)
prague.from_spe(random_spe, feature_list)

{'ATR': '+',
 'ant': '-',
 'approx': '+',
 'back': '+',
 'c.g.': '-',
 'cons': '0',
 'cont': '+',
 'coronal': '+',
 'del. rel.': '0',
 'dist': '0',
 'dorsal': '-',
 'front': '0',
 'high': '0',
 'labial': '0',
 'lat': '+',
 'low': '0',
 'nas': '0',
 'round': '+',
 's.g.': '-',
 'son': '-',
 'strid': '-',
 'syll': '0',
 'voice': '+'}

'[+ATR -ant +approx +back -c.g. +cont +coronal -dorsal +lat +round -s.g. -son -strid +voice]'

{'ATR': '+',
 'ant': '-',
 'approx': '+',
 'back': '+',
 'c.g.': '-',
 'cont': '+',
 'coronal': '+',
 'dorsal': '-',
 'lat': '+',
 'round': '+',
 's.g.': '-',
 'son': '-',
 'strid': '-',
 'voice': '+'}

{'ATR': '+',
 'ant': '-',
 'approx': '+',
 'back': '+',
 'c.g.': '-',
 'cons': '0',
 'cont': '+',
 'coronal': '+',
 'del. rel.': '0',
 'dist': '0',
 'dorsal': '-',
 'front': '0',
 'high': '0',
 'labial': '0',
 'lat': '+',
 'low': '0',
 'nas': '0',
 'round': '+',
 's.g.': '-',
 'son': '-',
 'strid': '-',
 'syll': '0',
 'voice': '+'}

## Combinatoric summary

In [25]:
f"|O| = {len(objects)} distinct object types (including labels)"
f"{unique_objects_np.shape[1]} object features"
"{0:.2E} logically possible object feature vectors".format(2**unique_objects_np.shape[1])
f"|V| = {unique_objects_np.shape[0]} distinct object feature vectors"
"{0:.2E} possible subsets of V".format(2**unique_objects_np.shape[0])
"{0:.2E} possible partial feature vectors".format(3**unique_objects_np.shape[1])

'|O| = 94 distinct object types (including labels)'

'23 object features'

'8.39E+06 logically possible object feature vectors'

'|V| = 91 distinct object feature vectors'

'2.48E+27 possible subsets of V'

'9.41E+10 possible partial feature vectors'

# Identifying a possible Boolean concept

Consider the two sets of objects defined below:

In [26]:
from random import choice

In [27]:
num_observations = 4
random_observation = [choice(objects) for each in range(num_observations)]
lmap(lambda o: o['symbol'],
     random_observation)
random_observation

['ɵ', 's', 'a', 'ø']

[OrderedDict([('symbol', 'ɵ'),
              ('syll', '+'),
              ('cons', '-'),
              ('son', '+'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '+'),
              ('voice', '+'),
              ('cont', '+'),
              ('del. rel.', '0'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '-'),
              ('low', '-'),
              ('back', '-'),
              ('front', '-'),
              ('approx', '+'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '+'),
              ('ATR', '+')]),
 OrderedDict([('symbol', 's'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '-'),
              ('coronal', '+'),
              ('dorsal', '-'),
              ('voice', '-'),
              ('cont', '+'),
              ('de

In [28]:
# matching_symbols = {'t','d','p','b','k','g'} #will probably consume ALL available memory a few cells downstream
matching_symbols = {'j','w'}
non_random_observation = lfilter(lambda o: o['symbol'] in matching_symbols,
                                 objects)
len(non_random_observation)
non_random_observation

2

[OrderedDict([('symbol', 'w'),
              ('syll', '-'),
              ('cons', '-'),
              ('son', '+'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '+'),
              ('voice', '+'),
              ('cont', '+'),
              ('del. rel.', '0'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '+'),
              ('low', '-'),
              ('back', '+'),
              ('front', '-'),
              ('approx', '+'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '+'),
              ('ATR', '0')]),
 OrderedDict([('symbol', 'j'),
              ('syll', '-'),
              ('cons', '-'),
              ('son', '+'),
              ('labial', '-'),
              ('coronal', '-'),
              ('dorsal', '+'),
              ('voice', '+'),
              ('cont', '+'),
              ('de

Suppose we didn't know how either set of observations were generated, but that we want to know if all of the examples in each set of observations are instances of at least one Boolean concept (i.e. plausibly generated by/instances of the same defining partial feature vector).

`prague`'s main functionality is to facilitate this kind of calculation and analysis via two functions:

In [29]:
print(prague.get_pfvs_whose_extension_contains.__doc__)


    Given
        a set of observed objects (a stack of feature vectors)
    this returns
        the set of partial feature vectors (a stack, one vector per row)
    whose extension must contain the set of observed objects.
    


In [30]:
print(prague.get_pfvs_whose_extension_is_exactly.__doc__)


    Given
        a set of observed objects (a stack of feature vectors)
        a set of potentially observable objects (another stack of vectors)
    this returns
        the set of partial feature vectors (a stack, one vector per row)
    whose extension must be exactly the set of observed objects.
    


To use these functions, we first (for each set of observations) map each object to its associated NumPy vector representation:

In [31]:
random_observation_pfvs = np.array([symbol_to_fv[o['symbol']]
                                   for o in random_observation])
random_observation_pfvs

array([[ 1,  0,  1, -1, -1, -1,  1, -1,  0,  0,  1, -1, -1,  1, -1, -1,
        -1,  1, -1,  1,  0,  1,  1],
       [ 0,  1, -1,  0, -1,  1,  1,  1,  1, -1, -1,  0,  0, -1, -1,  0,
        -1,  0, -1, -1,  1, -1, -1],
       [-1,  0,  1, -1, -1, -1,  1, -1,  0,  0,  1,  1, -1, -1, -1,  1,
        -1, -1, -1,  1,  0,  1,  1],
       [ 1,  0,  1, -1, -1, -1,  1, -1,  0,  0,  1,  1, -1,  1, -1, -1,
        -1,  1, -1,  1,  0,  1,  1]], dtype=int8)

Since the random observation set is random from run to run of the notebook, your specific results (if you execute this notebook locally) may vary, but in general, there will be thousands of partial feature vectors (= conjunctive normal form formulas = restricted Boolean concepts) that are compatible with (i.e. could have given rise to) this observation: 

In [32]:
possible_explanations_for_random_observation = prague.get_pfvs_whose_extension_contains(random_observation_pfvs)
possible_explanations_for_random_observation.shape
print(possible_explanations_for_random_observation)
for each_v in possible_explanations_for_random_observation:
    print(prague.to_spe(feature_list, each_v))

(32, 23)

[[ 0  0  0  0 -1  0  1  0  0  0  0  0  0  0 -1  0 -1  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0 -1  0 -1  0 -1  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1  0 -1  0 -1  0  0  0  0]
 [ 0  0  0  0 -1  0  1  0  0  0  0  0  0  0  0  0 -1  0 -1  0  0  0  0]
 [ 0  0  0  0 -1  0  1  0  0  0  0  0  0  0 -1  0  0  0 -1  0  0  0  0]
 [ 0  0  0  0 -1  0  1  0  0  0  0  0  0  0 -1  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0 -1  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -1  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0 -1  0  0  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0 -1  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0 -1  0 -1  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1  0  0  0 -1  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  1  0  0  0  0  0  0  0  0  0  0  0 -1  0  0

For example:

In [33]:
print(f"Recall: the random observation = \n{lmap(lambda o: o['symbol'], random_observation)}")
a_compatible_concept = choice(possible_explanations_for_random_observation)
print("A possible partial feature vector that could have given rise to this observation:\n"
      f"   {a_compatible_concept}\n"
      f" = {prague.to_spe(feature_list, a_compatible_concept)}"
#       "{0}".format({feature_list[i]:{-1:'-',1:'+'}[v] for i,v in enumerate(a_compatible_concept) if v in {1,-1}})
     )
      
compatible_concept_extension_vector = prague.extension(a_compatible_concept, 
                                                       unique_objects_np)
compatible_concept_extension_as_objects = prague.extension_vector_to_objects(compatible_concept_extension_vector, 
                                                                             unique_objects_np)
compatible_concept_extension_as_symbols = reduce(set.union, 
                                                 lmap(fv_to_symbols, 
                                                      compatible_concept_extension_as_objects), 
                                                 set())
      
print("The total set of observations that exhibit this concept:\n",
      f"{compatible_concept_extension_as_symbols}")

Recall: the random observation = 
['ɵ', 's', 'a', 'ø']
A possible partial feature vector that could have given rise to this observation:
   [ 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0]
 = [+cont -s.g.]
The total set of observations that exhibit this concept:
 {'ɛ', 'œ', 'ɥ', 'ɑ', 'χ', 'ɜ', 'f', 'ʃ', 'æ', 'ʌ', 's', 'ʙ', 'ɻ', 'ħ', 'ʊ', 'ɯ', 'ɨ', 'ʂ', 'ɹ', 'x', 'o', 'j', 'ʕ', 'ʐ', 'ʉ', 'ʏ', 'v', 'θ', 'ɤ', 'ø', 'z', 'ɵ', 'w', 'ɞ', 'e', 'ə', 'ð', 'ʁ', 'ɔ', 'ɒ', 'ɪ', 'ɘ', 'ʋ', 'β', 'u', 'ʀ', 'ɶ', 'a', 'y', 'ɣ', 'ɐ', 'ç', 'i', 'ɸ', 'ʒ', 'r', 'ʝ'}


In [34]:
precise_explanations_for_random_observation = prague.get_pfvs_whose_extension_is_exactly(random_observation_pfvs,
                                                                                         objects_np)
precise_explanations_for_random_observation.shape
print(f"{precise_explanations_for_random_observation} = ")
for each_v in precise_explanations_for_random_observation:
    print(prague.to_spe(feature_list, each_v))

#This will usually be empty: most subsets of the set of logically possible objects
# can't be described by a formula in conjunctive normal form

(0, 23)

[] = 


In [35]:
non_random_observation_pfvs = np.array([symbol_to_fv[o['symbol']]
                                        for o in non_random_observation])
print(non_random_observation_pfvs)
for each_v in non_random_observation_pfvs:
    print(prague.to_spe(feature_list, each_v))

[[ 0  0  1  1 -1 -1  1 -1  0  0  1 -1  1  1 -1 -1 -1  1 -1  1  0 -1  1]
 [ 0  0  1 -1 -1 -1  1 -1  0  0  1  1  1 -1 -1 -1 -1 -1 -1  1  0 -1  1]]
[+approx +back -c.g. -cons +cont -coronal +dorsal -front +high +labial -lat -low -nas +round -s.g. +son -syll +voice]
[+approx -back -c.g. -cons +cont -coronal +dorsal +front +high -labial -lat -low -nas -round -s.g. +son -syll +voice]


In [36]:
possible_explanations_for_non_random_observation = prague.get_pfvs_whose_extension_contains(non_random_observation_pfvs)
possible_explanations_for_non_random_observation.shape
print(possible_explanations_for_non_random_observation)
for each_v in possible_explanations_for_non_random_observation:
    print(prague.to_spe(feature_list, each_v))

(16384, 23)

[[ 0  0  1 ...  0 -1  1]
 [ 0  0  0 ...  0 -1  1]
 [ 0  0  1 ...  0 -1  1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
[+approx -c.g. -cons +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[-c.g. -cons +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -cons +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +dorsal -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +dorsal +high -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +dorsal +high -lat -nas -s.g. +son -syll +voice]
[+a

[+approx -c.g. -cons -coronal +dorsal +high -low -s.g. -syll +voice]
[+approx -c.g. -cons -coronal +dorsal +high -low -s.g. +son +voice]
[+approx -c.g. -cons -coronal +dorsal +high -low -s.g. +son -syll]
[+approx -c.g. -cons -coronal +dorsal +high -low -nas -syll +voice]
[+approx -c.g. -cons -coronal +dorsal +high -low -nas +son +voice]
[+approx -c.g. -cons -coronal +dorsal +high -low -nas +son -syll]
[+approx -c.g. -cons -coronal +dorsal +high -low -nas -s.g. +voice]
[+approx -c.g. -cons -coronal +dorsal +high -low -nas -s.g. -syll]
[+approx -c.g. -cons -coronal +dorsal +high -low -nas -s.g. +son]
[+approx -c.g. -cons -coronal +dorsal +high -lat +son -syll +voice]
[+approx -c.g. -cons -coronal +dorsal +high -lat -s.g. -syll +voice]
[+approx -c.g. -cons -coronal +dorsal +high -lat -s.g. +son +voice]
[+approx -c.g. -cons -coronal +dorsal +high -lat -s.g. +son -syll]
[+approx -c.g. -cons -coronal +dorsal +high -lat -nas -syll +voice]
[+approx -c.g. -cons -coronal +dorsal +high -lat -nas 

[+approx -c.g. -cons +cont -coronal -lat -low -s.g. +son -syll]
[+approx -c.g. -cons +cont -coronal -lat -low -nas -syll +voice]
[+approx -c.g. -cons +cont -coronal -lat -low -nas +son +voice]
[+approx -c.g. -cons +cont -coronal -lat -low -nas +son -syll]
[+approx -c.g. -cons +cont -coronal -lat -low -nas -s.g. +voice]
[+approx -c.g. -cons +cont -coronal -lat -low -nas -s.g. -syll]
[+approx -c.g. -cons +cont -coronal -lat -low -nas -s.g. +son]
[+approx -c.g. -cons +cont -coronal +high -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +high -nas +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +high -nas -s.g. -syll +voice]
[+approx -c.g. -cons +cont -coronal +high -nas -s.g. +son +voice]
[+approx -c.g. -cons +cont -coronal +high -nas -s.g. +son -syll]
[+approx -c.g. -cons +cont -coronal +high -low +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +high -low -s.g. -syll +voice]
[+approx -c.g. -cons +cont -coronal +high -low -s.g. +son +voice]
[+approx -c.g. -cons +

[-cons +cont -coronal +dorsal +high -low +son -syll +voice]
[-cons +cont -coronal +dorsal +high -low -s.g. -syll +voice]
[-cons +cont -coronal +dorsal +high -low -s.g. +son +voice]
[-cons +cont -coronal +dorsal +high -low -s.g. +son -syll]
[-cons +cont -coronal +dorsal +high -low -nas -syll +voice]
[-cons +cont -coronal +dorsal +high -low -nas +son +voice]
[-cons +cont -coronal +dorsal +high -low -nas +son -syll]
[-cons +cont -coronal +dorsal +high -low -nas -s.g. +voice]
[-cons +cont -coronal +dorsal +high -low -nas -s.g. -syll]
[-cons +cont -coronal +dorsal +high -low -nas -s.g. +son]
[-cons +cont -coronal +dorsal +high -lat +son -syll +voice]
[-cons +cont -coronal +dorsal +high -lat -s.g. -syll +voice]
[-cons +cont -coronal +dorsal +high -lat -s.g. +son +voice]
[-cons +cont -coronal +dorsal +high -lat -s.g. +son -syll]
[-cons +cont -coronal +dorsal +high -lat -nas -syll +voice]
[-cons +cont -coronal +dorsal +high -lat -nas +son +voice]
[-cons +cont -coronal +dorsal +high -lat -nas +

...

[+cont +high -lat -s.g. +son +voice]
[+cont +high -lat -s.g. +son -syll]
[+cont +high -lat -nas -syll +voice]
[+cont +high -lat -nas +son +voice]
[+cont +high -lat -nas +son -syll]
[+cont +high -lat -nas -s.g. +voice]
[+cont +high -lat -nas -s.g. -syll]
[+cont +high -lat -nas -s.g. +son]
[+cont +high -lat -low -syll +voice]
[+cont +high -lat -low +son +voice]
[+cont +high -lat -low +son -syll]
[+cont +high -lat -low -s.g. +voice]
[+cont +high -lat -low -s.g. -syll]
[+cont +high -lat -low -s.g. +son]
[+cont +high -lat -low -nas +voice]
[+cont +high -lat -low -nas -syll]
[+cont +high -lat -low -nas +son]
[+cont +high -lat -low -nas -s.g.]
[+cont +dorsal -s.g. +son -syll +voice]
[+cont +dorsal -nas +son -syll +voice]
[+cont +dorsal -nas -s.g. -syll +voice]
[+cont +dorsal -nas -s.g. +son +voice]
[+cont +dorsal -nas -s.g. +son -syll]
[+cont +dorsal -low +son -syll +voice]
[+cont +dorsal -low -s.g. -syll +voice]
[+cont +dorsal -low -s.g. +son +voice]
[+cont +dorsal -low -s.g. +son -syll]
[+c

[-c.g. +cont +high -lat -nas -syll]
[-c.g. +cont +high -lat -nas +son]
[-c.g. +cont +high -lat -nas -s.g.]
[-c.g. +cont +high -lat -low +voice]
[-c.g. +cont +high -lat -low -syll]
[-c.g. +cont +high -lat -low +son]
[-c.g. +cont +high -lat -low -s.g.]
[-c.g. +cont +high -lat -low -nas]
[-c.g. +cont +dorsal +son -syll +voice]
[-c.g. +cont +dorsal -s.g. -syll +voice]
[-c.g. +cont +dorsal -s.g. +son +voice]
[-c.g. +cont +dorsal -s.g. +son -syll]
[-c.g. +cont +dorsal -nas -syll +voice]
[-c.g. +cont +dorsal -nas +son +voice]
[-c.g. +cont +dorsal -nas +son -syll]
[-c.g. +cont +dorsal -nas -s.g. +voice]
[-c.g. +cont +dorsal -nas -s.g. -syll]
[-c.g. +cont +dorsal -nas -s.g. +son]
[-c.g. +cont +dorsal -low -syll +voice]
[-c.g. +cont +dorsal -low +son +voice]
[-c.g. +cont +dorsal -low +son -syll]
[-c.g. +cont +dorsal -low -s.g. +voice]
[-c.g. +cont +dorsal -low -s.g. -syll]
[-c.g. +cont +dorsal -low -s.g. +son]
[-c.g. +cont +dorsal -low -nas +voice]
[-c.g. +cont +dorsal -low -nas -syll]
[-c.g. +c

[+approx +high -low -nas +son +voice]
[+approx +high -low -nas +son -syll]
[+approx +high -low -nas -s.g. +voice]
[+approx +high -low -nas -s.g. -syll]
[+approx +high -low -nas -s.g. +son]
[+approx +high -lat +son -syll +voice]
[+approx +high -lat -s.g. -syll +voice]
[+approx +high -lat -s.g. +son +voice]
[+approx +high -lat -s.g. +son -syll]
[+approx +high -lat -nas -syll +voice]
[+approx +high -lat -nas +son +voice]
[+approx +high -lat -nas +son -syll]
[+approx +high -lat -nas -s.g. +voice]
[+approx +high -lat -nas -s.g. -syll]
[+approx +high -lat -nas -s.g. +son]
[+approx +high -lat -low -syll +voice]
[+approx +high -lat -low +son +voice]
[+approx +high -lat -low +son -syll]
[+approx +high -lat -low -s.g. +voice]
[+approx +high -lat -low -s.g. -syll]
[+approx +high -lat -low -s.g. +son]
[+approx +high -lat -low -nas +voice]
[+approx +high -lat -low -nas -syll]
[+approx +high -lat -low -nas +son]
[+approx +high -lat -low -nas -s.g.]
[+approx +dorsal -s.g. +son -syll +voice]
[+approx 

[+cont +dorsal -lat -s.g. +son]
[+cont +dorsal -lat -nas +voice]
[+cont +dorsal -lat -nas -syll]
[+cont +dorsal -lat -nas +son]
[+cont +dorsal -lat -nas -s.g.]
[+cont +dorsal -lat -low +voice]
[+cont +dorsal -lat -low -syll]
[+cont +dorsal -lat -low +son]
[+cont +dorsal -lat -low -s.g.]
[+cont +dorsal -lat -low -nas]
[+cont +dorsal +high -syll +voice]
[+cont +dorsal +high +son +voice]
[+cont +dorsal +high +son -syll]
[+cont +dorsal +high -s.g. +voice]
[+cont +dorsal +high -s.g. -syll]
[+cont +dorsal +high -s.g. +son]
[+cont +dorsal +high -nas +voice]
[+cont +dorsal +high -nas -syll]
[+cont +dorsal +high -nas +son]
[+cont +dorsal +high -nas -s.g.]
[+cont +dorsal +high -low +voice]
[+cont +dorsal +high -low -syll]
[+cont +dorsal +high -low +son]
[+cont +dorsal +high -low -s.g.]
[+cont +dorsal +high -low -nas]
[+cont +dorsal +high -lat +voice]
[+cont +dorsal +high -lat -syll]
[+cont +dorsal +high -lat +son]
[+cont +dorsal +high -lat -s.g.]
[+cont +dorsal +high -lat -nas]
[+cont +dorsal +h

[+cont -s.g. -syll +voice]
[+cont -s.g. +son +voice]
[+cont -s.g. +son -syll]
[+cont -nas -syll +voice]
[+cont -nas +son +voice]
[+cont -nas +son -syll]
[+cont -nas -s.g. +voice]
[+cont -nas -s.g. -syll]
[+cont -nas -s.g. +son]
[+cont -low -syll +voice]
[+cont -low +son +voice]
[+cont -low +son -syll]
[+cont -low -s.g. +voice]
[+cont -low -s.g. -syll]
[+cont -low -s.g. +son]
[+cont -low -nas +voice]
[+cont -low -nas -syll]
[+cont -low -nas +son]
[+cont -low -nas -s.g.]
[+cont -lat -syll +voice]
[+cont -lat +son +voice]
[+cont -lat +son -syll]
[+cont -lat -s.g. +voice]
[+cont -lat -s.g. -syll]
[+cont -lat -s.g. +son]
[+cont -lat -nas +voice]
[+cont -lat -nas -syll]
[+cont -lat -nas +son]
[+cont -lat -nas -s.g.]
[+cont -lat -low +voice]
[+cont -lat -low -syll]
[+cont -lat -low +son]
[+cont -lat -low -s.g.]
[+cont -lat -low -nas]
[+cont +high -syll +voice]
[+cont +high +son +voice]
[+cont +high +son -syll]
[+cont +high -s.g. +voice]
[+cont +high -s.g. -syll]
[+cont +high -s.g. +son]
[+con

In [37]:
precise_explanations_for_non_random_observation = prague.get_pfvs_whose_extension_is_exactly(non_random_observation_pfvs,
                                                                                             objects_np)
precise_explanations_for_non_random_observation.shape
print(precise_explanations_for_non_random_observation)
for each_v in precise_explanations_for_non_random_observation:
    print(prague.to_spe(feature_list, each_v))

(4672, 23)

[[ 0  0  1 ...  0 -1  1]
 [ 0  0  0 ...  0 -1  1]
 [ 0  0  1 ...  0 -1  1]
 ...
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0 -1  0]
 [ 0  0  0 ...  0 -1  0]]
[+approx -c.g. -cons +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[-c.g. -cons +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -cons +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. +cont -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons -coronal +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont +dorsal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +high -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +dorsal -lat -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +dorsal +high -low -nas -s.g. +son -syll +voice]
[+approx -c.g. -cons +cont -coronal +dorsal +high -lat -nas -s.g. +son -syll +voice]
[+a

[-c.g. -cons +cont +dorsal -lat -s.g. -syll +voice]
[-c.g. -cons +cont +dorsal -lat -s.g. +son -syll]
[-c.g. -cons +cont +dorsal -lat -nas -syll +voice]
[-c.g. -cons +cont +dorsal -lat -nas +son -syll]
[-c.g. -cons +cont +dorsal -lat -nas -s.g. -syll]
[-c.g. -cons +cont +dorsal -lat -low -syll +voice]
[-c.g. -cons +cont +dorsal -lat -low +son -syll]
[-c.g. -cons +cont +dorsal -lat -low -s.g. -syll]
[-c.g. -cons +cont +dorsal -lat -low -nas -syll]
[-c.g. -cons +cont +dorsal +high +son -syll +voice]
[-c.g. -cons +cont +dorsal +high -s.g. -syll +voice]
[-c.g. -cons +cont +dorsal +high -s.g. +son -syll]
[-c.g. -cons +cont +dorsal +high -nas -syll +voice]
[-c.g. -cons +cont +dorsal +high -nas +son -syll]
[-c.g. -cons +cont +dorsal +high -nas -s.g. -syll]
[-c.g. -cons +cont +dorsal +high -low -syll +voice]
[-c.g. -cons +cont +dorsal +high -low +son -syll]
[-c.g. -cons +cont +dorsal +high -low -s.g. -syll]
[-c.g. -cons +cont +dorsal +high -low -nas -syll]
[-c.g. -cons +cont +dorsal +high -lat

[-c.g. -cons +dorsal -nas +son -syll]
[-c.g. -cons +dorsal -nas -s.g. -syll]
[-c.g. -cons +dorsal -low -syll +voice]
[-c.g. -cons +dorsal -low +son -syll]
[-c.g. -cons +dorsal -low -s.g. -syll]
[-c.g. -cons +dorsal -low -nas -syll]
[-c.g. -cons +dorsal -lat -syll +voice]
[-c.g. -cons +dorsal -lat +son -syll]
[-c.g. -cons +dorsal -lat -s.g. -syll]
[-c.g. -cons +dorsal -lat -nas -syll]
[-c.g. -cons +dorsal -lat -low -syll]
[-c.g. -cons +dorsal +high -syll +voice]
[-c.g. -cons +dorsal +high +son -syll]
[-c.g. -cons +dorsal +high -s.g. -syll]
[-c.g. -cons +dorsal +high -nas -syll]
[-c.g. -cons +dorsal +high -low -syll]
[-c.g. -cons +dorsal +high -lat -syll]
[-c.g. -cons -coronal -low -syll +voice]
[-c.g. -cons -coronal -low +son -syll]
[-c.g. -cons -coronal -low -s.g. -syll]
[-c.g. -cons -coronal -low -nas -syll]
[-c.g. -cons -coronal -lat -low -syll]
[-c.g. -cons -coronal +high -syll +voice]
[-c.g. -cons -coronal +high +son -syll]
[-c.g. -cons -coronal +high -s.g. -syll]
[-c.g. -cons -cor

In [38]:
degree_of_specification_of_exact_matches = np.abs(precise_explanations_for_non_random_observation).sum(axis=1)
minimal_specification_of_exact_matches = np.min(degree_of_specification_of_exact_matches)
print(f"# features specified in exact matches = {minimal_specification_of_exact_matches}")
minimal_pfvs_that_are_exact_matches = precise_explanations_for_non_random_observation[degree_of_specification_of_exact_matches == minimal_specification_of_exact_matches]
print(f"# minimal pfvs that are exact matches = {minimal_pfvs_that_are_exact_matches.shape[0]}")
print(f"Minimal pfvs that are exact matches =\n"
      f"{minimal_pfvs_that_are_exact_matches}")
for each_v in minimal_pfvs_that_are_exact_matches:
    print(prague.to_spe(feature_list, each_v))

# for each_pfv in minimal_pfvs_that_are_exact_matches:
#     print(pfv_to_fd(each_pfv))

print("Sanity check: extensions of the three pfvs = ")
for each_pfv in minimal_pfvs_that_are_exact_matches:
    each_ext = prague.extension(each_pfv, unique_objects_np)
    reduce(set.union, extension_to_symbols(each_ext), set())

# features specified in exact matches = 3
# minimal pfvs that are exact matches = 3
Minimal pfvs that are exact matches =
[[ 0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0 -1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0 -1  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 -1  0]]
[-cons -low -syll]
[-cons +high -syll]
[-cons +dorsal -syll]
Sanity check: extensions of the three pfvs = 


{'j', 'w', 'ɥ'}

{'j', 'w', 'ɥ'}

{'j', 'w', 'ɥ'}

# General case

In [39]:
from tqdm import tqdm

In [40]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           125G        696M        118G        3.0M        7.0G        124G
Swap:          2.0G          0B        2.0G


In [41]:
#7s on wittgenstein
lower_closures = [prague.lower_closure(o, strict=False) 
                  for o in tqdm(unique_objects_np)]

100%|██████████| 91/91 [00:05<00:00,  8.64it/s]


In [42]:
lower_closures_as_matrix = np.concatenate(lower_closures)

In [43]:
#2m on wittgenstein
all_pfvs_with_nonempty_extension = np.unique(lower_closures_as_matrix, 
                                             return_index=False,
                                             axis=0)

In [44]:
all_pfvs_with_nonempty_extension.shape

(9115112, 23)

In [45]:
print("# partial feature vectors that pick out a non-empty subset of O: "
      "{0:.2E}\n".format(all_pfvs_with_nonempty_extension.shape[0]),
      "# logically possible partial feature vectors: "
      "{0:.2E}".format(3**all_pfvs_with_nonempty_extension.shape[1]))

# partial feature vectors that pick out a non-empty subset of O: 9.12E+06
 # logically possible partial feature vectors: 9.41E+10


In [46]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           125G        1.9G        116G        3.0M        7.0G        122G
Swap:          2.0G          0B        2.0G


In [47]:
# 2m on wittgenstein = no time complexity savings relative to np.unique
# unique_lower_closures = set(lmap(prague.HashableArray,
#                                  list(lower_closures_as_matrix)))

In [48]:
# !free -h

In [49]:
# >>>2m on wittgenstein, uses way too much memory and definitey no savings over
# np.unique
# unique_lower_closures2 = set(lmap(prague.HashableArray,
#                                   cat(map(list, lower_closures_as_matrix))))

In [50]:
# !free -h

In [51]:
#48s on wittgenstein
nonempty_pfv_extensions = prague.extensions(all_pfvs_with_nonempty_extension,
                                            object_inventory=unique_objects_np)

In [52]:
nonempty_pfv_extensions.shape

(9115112, 91)

In [53]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           125G        2.7G        116G        3.0M        7.0G        122G
Swap:          2.0G          0B        2.0G


## Demo

In [54]:
a_random_pfv = prague.feature_vector.make_random_pfv(num_features=len(feature_list))
random_extension = prague.extension(a_random_pfv, 
                                    object_inventory=unique_objects_np)
random_objects = prague.extension_vector_to_objects(random_extension, 
                                                    object_inventory=unique_objects_np)
random_symbols = reduce(set.union, extension_to_symbols(random_extension), set())
my_x = random_extension
my_S = random_objects
my_symbs = random_symbols
my_fd = pfv_to_fd(a_random_pfv)
my_symbs_fds = [pfv_to_fd(pfv) for pfv in my_S]


while my_x.sum() == 0:
    a_random_pfv = prague.feature_vector.make_random_pfv(num_features=len(feature_list))
    random_extension = prague.extension(a_random_pfv, 
                                        object_inventory=unique_objects_np)
    random_objects = prague.extension_vector_to_objects(random_extension, 
                                                        object_inventory=unique_objects_np)
    random_symbols = reduce(set.union, extension_to_symbols(random_extension), set())
    my_x = random_extension
    my_S = random_objects
    my_symbs = random_symbols
    my_fd = pfv_to_fd(a_random_pfv)
    my_symbs_fds = [pfv_to_fd(pfv) for pfv in my_S]


print(f"My random PFV:\n{a_random_pfv}\n{prague.to_spe(feature_list, a_random_pfv)}")
# print(f"My random PFV as a feature dict =\n{my_fd}")
print(f"My extension as an 'indicator' vector = \n{my_x}")
print(f"My extension as a stack of {my_S.shape[0]} objects S = \n{my_S}")
print(f"My extension as a set of matching symbols = \n{my_symbs}")
print(f"My extension in SPE-style feature notation:")
for each_v in my_S:
    print(prague.to_spe(feature_list, each_v))
# print(f"My extension as a set of feature dicts = \n",
#       my_symbs_fds)

My random PFV:
[ 0  0 -1  1 -1  1  1  0  0  0  1 -1 -1  0 -1  0  0  0 -1 -1  0 -1  0]
[-approx +back -c.g. +cons +cont +dorsal -front -high -lat -s.g. -son -syll]
My extension as an 'indicator' vector = 
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
My extension as a stack of 4 objects S = 
[[-1  0 -1  1 -1  1  1 -1  1  0  1 -1 -1 -1 -1  1 -1  0 -1 -1  0 -1 -1]
 [-1  0 -1  1 -1  1  1 -1  1  0  1 -1 -1 -1 -1  1 -1  0 -1 -1  0 -1  1]
 [ 0  0 -1  1 -1  1  1 -1  1  0  1 -1 -1 -1 -1 -1 -1  0 -1 -1  0 -1 -1]
 [ 0  0 -1  1 -1  1  1 -1  1  0  1 -1 -1 -1 -1 -1 -1  0 -1 -1  0 -1  1]]
My extension as a set of matching symbols = 
{'χ', 'ʁ', 'ʕ', 'ħ'}
My extension in SPE-style feature notation:
[-ATR -approx +back -c.g. +cons +cont -coronal +del. rel. +dorsal -front -high -labial -lat +low -nas -s.g. -son -syll -voice]
[-ATR -approx +back -c.g. +cons +cont -cor

In [55]:
pfvs_containing_my_S = prague.get_pfvs_whose_extension_contains(my_S)
pfvs_exactly_matching_my_S = prague.get_pfvs_whose_extension_is_exactly(my_S, 
                                                                        object_inventory=unique_objects_np)
specification_of_exact_matches = np.abs(pfvs_exactly_matching_my_S).sum(axis=1)
minimal_specification = np.min(specification_of_exact_matches)
minimal_pfvs_exactly_matching_my_s = pfvs_exactly_matching_my_S[specification_of_exact_matches == minimal_specification]

print("{0:,} PFVs".format(pfvs_containing_my_S.shape[0]),
      f"whose extension contains S = \n{pfvs_containing_my_S}")
print("{0:,} PFVs".format(pfvs_exactly_matching_my_S.shape[0]),
      f"whose extension is exactly S = \n{pfvs_exactly_matching_my_S}")
print("{0:,} PFVs".format(minimal_pfvs_exactly_matching_my_s.shape[0]),
      f"whose extension is exactly S and which are maximally simple"
      f" (i.e. unspecified) = \n{minimal_pfvs_exactly_matching_my_s}\n")
for each_v in minimal_pfvs_exactly_matching_my_s:
    print(prague.to_spe(feature_list, each_v))
print(f"original generating pfv = \n {a_random_pfv}\n "
      f"{prague.to_spe(feature_list, a_random_pfv)}")

65,536 PFVs whose extension contains S = 
[[ 0  0 -1 ...  0 -1  0]
 [ 0  0  0 ...  0 -1  0]
 [ 0  0 -1 ...  0 -1  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0 -1 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
22,528 PFVs whose extension is exactly S = 
[[ 0  0 -1 ...  0 -1  0]
 [ 0  0  0 ...  0 -1  0]
 [ 0  0 -1 ...  0 -1  0]
 ...
 [ 0  0 -1 ...  0  0  0]
 [ 0  0 -1 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
1 PFVs whose extension is exactly S and which are maximally simple (i.e. unspecified) = 
[[ 0  0  0  0  0  0  0  0  1  0  0  0 -1  0  0  0  0  0  0  0  0  0  0]]

[+del. rel. -high]
original generating pfv = 
 [ 0  0 -1  1 -1  1  1  0  0  0  1 -1 -1  0 -1  0  0  0 -1 -1  0 -1  0]
 [-approx +back -c.g. +cons +cont +dorsal -front -high -lat -s.g. -son -syll]


In [56]:
print(prague.extension(a_random_pfv, unique_objects_np))
for each_exact_match in minimal_pfvs_exactly_matching_my_s:
    print(prague.extension(each_exact_match, unique_objects_np))

[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [57]:
extension_to_symbols(prague.extension(a_random_pfv, unique_objects_np))
for each_exact_match in minimal_pfvs_exactly_matching_my_s:
    extension_to_symbols(prague.extension(each_exact_match, unique_objects_np))

array([{'ħ'}, {'ʕ'}, {'χ'}, {'ʁ'}], dtype=object)

array([{'ħ'}, {'ʕ'}, {'χ'}, {'ʁ'}], dtype=object)

In [58]:
a_random_pfv
prague.to_spe(feature_list, a_random_pfv)

array([ 0,  0, -1,  1, -1,  1,  1,  0,  0,  0,  1, -1, -1,  0, -1,  0,  0,
        0, -1, -1,  0, -1,  0], dtype=int8)

'[-approx +back -c.g. +cons +cont +dorsal -front -high -lat -s.g. -son -syll]'

In [59]:
print('Specified indices to modify:')
specified_index_a = choice(list(a_random_pfv.nonzero()[0]))
specified_index_a
specified_index_b = choice(list(a_random_pfv.nonzero()[0]))
while specified_index_b == specified_index_a:
    specified_index_b = choice(list(a_random_pfv.nonzero()[0]))
specified_index_b

print('\nModification vector:')
modification = np.zeros(a_random_pfv.shape, dtype=np.int8)
modification[specified_index_a] = choice([-1, 1])
modification[specified_index_b] = choice([-1, 1])
modification
prague.to_spe(feature_list, modification)

print('\nNaive update:')
updated_naive = prague.feature_vector.spe_update(a_random_pfv, modification)
updated_naive
prague.to_spe(feature_list, updated_naive)

updated = prague.feature_vector.spe_update(a_random_pfv, modification, unique_objects_np)
print('\nMinimally further coerced vectors with non-empty extension:\n'
      f'{updated}')
print('\nCoerced updates:')
if updated.ndim == 1:
    print('No further coercion necessary:')
    print(prague.to_spe(feature_list, updated))
else:
    for each in updated:
        print(prague.to_spe(feature_list, each))

Specified indices to modify:


10

19


Modification vector:


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0], dtype=int8)

'[+dorsal +son]'


Naive update:


array([ 0,  0, -1,  1, -1,  1,  1,  0,  0,  0,  1, -1, -1,  0, -1,  0,  0,
        0, -1,  1,  0, -1,  0], dtype=int8)

'[-approx +back -c.g. +cons +cont +dorsal -front -high -lat -s.g. +son -syll]'

Coerced vector has empty extension:
[ 0  0 -1  1 -1  1  1  0  0  0  1 -1 -1  0 -1  0  0  0 -1  1  0 -1  0]

Minimally further coerced vectors with non-empty extension:
[[ 0  0  0  1 -1  1  1  0  0  0  1 -1 -1  0 -1  0  0  0 -1  1  0 -1  0]
 [ 0  0 -1  1 -1  1  0  0  0  0  1 -1 -1  0 -1  0  0  0 -1  1  0 -1  0]
 [ 0  0 -1  1 -1  1 -1  0  0  0  1 -1 -1  0 -1  0  0  0 -1  1  0 -1  0]]

Coerced updates:
[+back -c.g. +cons +cont +dorsal -front -high -lat -s.g. +son -syll]
[-approx +back -c.g. +cons +dorsal -front -high -lat -s.g. +son -syll]
[-approx +back -c.g. +cons -cont +dorsal -front -high -lat -s.g. +son -syll]
